In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from helper import *

In [2]:
df = pd.read_csv('../data/train.csv')
#df.head()

In [3]:
df.drop(['Id'], axis=1, inplace=True)

In [4]:
categoric_df = df.select_dtypes(exclude=[np.number])
categoric_df['SalePrice'] = df['SalePrice']
print(categoric_df.columns)

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='object')


In [5]:
categoric_df.drop(['MiscFeature'], axis=1, inplace=True)

'MiscFeature' is the feature of the house not covered in other categories but their is a feature called 'MiscVal' which is the value of the 'MiscFeature'. So, we can drop this feature as it is just a description but the value is already accounted for in numeric_df

In [6]:
count_nulls(categoric_df)

,nulls,percent_missing
PoolQC,1453,0.995205
Alley,1369,0.937671
Fence,1179,0.807534
MasVnrType,872,0.597260
FireplaceQu,690,0.472603
GarageType,81,0.055479
GarageCond,81,0.055479
GarageQual,81,0.055479
GarageFinish,81,0.055479
BsmtExposure,38,0.026027


There are a lot of nulls for Pools, Alleys, Fences, etc in the dataset. However in the description **NA** means No Pool, No Alley, No Fence, etc. So I will replace certain nulls with descriptions in the notes with (str) **'none'** before onehot encoding.

In [7]:
na_pairings = {'PoolQC' : 'No Pool',
               'Alley' : 'No Alley',
                'Fence' : 'No Fence',
                'FireplaceQu' : 'No Fireplace',
                'GarageType' : 'No Garage',
                'GarageFinish' : 'No Garage',
                'GarageQual' : 'No Garage',
                'GarageCond' : 'No Garage',
                'BsmtExposure' : 'No Basement',
                'BsmtFinType1' : 'No Basement',
                'BsmtFinType2' : 'No Basement',
                'BsmtQual' : 'No Basement',
                'BsmtCond' : 'No Basement',        
                }

#electircal should not be na, drop na electorcal row
categoric_df.dropna(subset=['Electrical'], inplace=True)

In [8]:
#remove masonry veneer type col
categoric_df.drop(['MasVnrType'], axis=1, inplace=True)

In [ ]:
#apply one hot encoding
categoric_df = pd.get_dummies(categoric_df, drop_first=True)


,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,SaleType,SaleCondition,SalePrice
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,WD,Normal,208500
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,WD,Normal,181500
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,WD,Normal,223500
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,WD,Abnorml,140000
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,WD,Normal,175000
1456,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,WD,Normal,210000
1457,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,WD,Normal,266500
1458,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,NaN,WD,Normal,142125
